#Проект по аналитике опубликованных вакансий на LinkedIn

##Цели и задачи

**Исходные данные:** таблица csv с результатом парсинга вакансий LinkedIn в HTML-формате.

**Цель:** визуализировать информацию о рынке вакансий для аналитиков в Европе.

**Задачи:**

- распарсить предоставленный csv файл создав следующие
признаки:
    - наименование вакансии
    - город;|
    - страна;
    - тип занятости;
    - компания;
    - размер компании (количество работников);
    - сфера деятельности компании;
    - требуемые хард скилы;
    - дата публикации вакансии;
    - количество кандидатов на вакансию.

- проверить данные на дубликаты и не релевантные заданию вакансии.
- построить интерактивный дашборд в BI системе.

##Загрузка данных

In [ ]:
# испортируем библиотеки

import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import csv

from IPython.display import display, HTML
pd.set_option("max_colwidth", 100)
pd.set_option("display.max_rows", None)

from datetime import datetime, timedelta

from warnings import filterwarnings
pd.options.mode.chained_assignment = None
from tqdm.notebook import tqdm

In [ ]:
# загрузим файл

from google.colab import files

data = files.upload()

Saving masterskaya_parsing_LinkedIn_2023_05_23.csv to masterskaya_parsing_LinkedIn_2023_05_23.csv


In [ ]:
# загрузим данные в переменную df и выведем первые 5 строк датафрейма

df = pd.read_csv('masterskaya_parsing_LinkedIn_2023_05_23.csv')
df.head(5)

,Unnamed: 0,html
0,0,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing..."
1,1,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing..."
2,2,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing..."
3,3,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing..."
4,4,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing..."


In [ ]:
# посмотрим, как выглядела бы вакансия на сайте

display(HTML(df['html'][192]))

##Получение данных

###Наименование вакансии

In [ ]:
# загрузим данные по наименованию вакансии в столбец name

df['name'] = [BeautifulSoup(df['html'][x]).find('h2').text for x in tqdm(range(len(df['html'])))]


  0%|          | 0/998 [00:00<?, ?it/s]

In [ ]:
df.head()

,Unnamed: 0,html,name
0,0,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing...",Data Analyst
1,1,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing...",Data Analyst - Logistics
2,2,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing...",Data Analyst - Logistics
3,3,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing...",Data Analyst (Space & Planning)
4,4,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing...",Data Analyst


###Город и страна

Чтобы получить данные по городу и стране, предварительно создадим столбец с полными данными о локации.

In [ ]:
df['area'] = np.nan

df['area'] = [BeautifulSoup(df['html'][x]).find('span', class_='jobs-unified-top-card__bullet').text.strip() for x in tqdm(range(len(df['html'])))]



  0%|          | 0/998 [00:00<?, ?it/s]

In [ ]:
df.head()

,Unnamed: 0,html,name,area
0,0,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing...",Data Analyst,"Basel, Basel, Switzerland"
1,1,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing...",Data Analyst - Logistics,"Coventry, England, United Kingdom"
2,2,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing...",Data Analyst - Logistics,"Coventry, England, United Kingdom"
3,3,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing...",Data Analyst (Space & Planning),"South Molton, England, United Kingdom"
4,4,"\n <div>\n <div class=""\n jobs-details__main-content jobs-details__main-content--sing...",Data Analyst,"Lugano, Ticino, Switzerland"


In [ ]:
# разделим значения столбца area на столбцы с городом, регионом и страной

df[['city', 'location', 'country']] = df['area'].str.split(pat=',', expand=True)

Для более точного указания стран, создадим список стран мира, а потом проверим данные на соответствие ему.

In [ ]:
# создадим список стран мира

countries = ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
             'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
             'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
             'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
             'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
             'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde',
             'Cambodia', 'Cameroon', 'Canada', 'Central African Republic',
             'Chad', 'Chile', 'China', 'Colombia', 'Comoros',
             'Congo, Democratic Republic of the',
             'Congo, Republic of the', 'Costa Rica',
             "Cote d'Ivoire", 'Croatia', 'Cuba',
             'Cyprus',
             'Czech Republic', 'Czechia',
             'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic',
             'East Timor (Timor-Leste)', 'Ecuador', 'Egypt', 'El Salvador',
             'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini (formerly Swaziland)',
             'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany',
             'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea',
             'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary',
             'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq',
             'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan',
             'Kazakhstan', 'Kenya', 'Kiribati', 'Kosovo', 'Kuwait',
             'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho',
             'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Madagascar',
             'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Mauritania', 'Mauritius',
             'Mexico', 'Micronesia', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique',
             'Myanmar (formerly Burma)', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Zealand',
             'Nicaragua', 'Niger', 'Nigeria', 'North Korea', 'North Macedonia (formerly Macedonia)',
             'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestine State', 'Panama',
             'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal',
             'Qatar', 'Romania', 'Russia', 'Rwanda', 'Saint Kitts and Nevis',
             'Saint Lucia', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino',
             'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles',
             'Sierra Leone', 'Singapore', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Korea',
             'South Sudan', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'Sweden', 'Switzerland',
             'Syria', 'Taiwan', 'Tajikistan', 'Tanzania', 'Thailand', 'Tibet', 'Togo', 'Tonga', 'Trinidad and Tobago', 'Tunisia',
             'Turkey', 'Turkmenistan', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom',
             'United States of America', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Vatican City (Holy See)', 'Venezuela', 'Vietnam', 'Yemen', 'Zambia', 'Zimbabwe']

Создадим столбец country, в который будем добавлять страну по соответствию ее упоминания в столбце area нашему списку стран мира.

In [ ]:
res_country = []

for index in tqdm(range(len(df['html']))):
  country_list = []
  for country in countries:
    if country in df['area'][index]:
      country_list.append(country)
  res_country.append(country_list)

df['country'] = res_country
df['country'] = df['country'].apply(lambda x: ''.join(x))


  0%|          | 0/998 [00:00<?, ?it/s]

###Тип занятости

In [ ]:
df['employment_type'] = np.nan

for x in tqdm(range(len(df['html']))):
  try:
    df['employment_type'][x] = BeautifulSoup(df['html'][x]).find('span', class_='jobs-unified-top-card__workplace-type').text
  except:
    continue


  0%|          | 0/998 [00:00<?, ?it/s]

###Название компании

In [ ]:
# соберем в столбец company_title названия компаний

df['company_title'] = np.nan

for x in tqdm(range(len(df['html']))):
  try:
    df['company_title'][x] = BeautifulSoup(df['html'][x]).find('div', class_ = 'jobs-unified-top-card__primary-description').find('a').text.strip()
  except:
    continue



  0%|          | 0/998 [00:00<?, ?it/s]

### Размер компаний

In [ ]:
# соберем в столбец company_size данные о размере компании

df['company_size'] = np.nan

for x in tqdm(range(len(df['html']))):
  try:
    df['company_size'][x] = BeautifulSoup(df['html'][x]).find('div', class_='mt5 mb2').find_all('span')[1].text.split()[0]
  except:
    continue





  0%|          | 0/998 [00:00<?, ?it/s]

###Сфера деятельности компаний

In [ ]:
# соберем в столбец industry информацию о сфере деятельности компании

df['industry'] = np.nan

for x in tqdm(range(len(df['html']))):
  try:
    df['industry'][x] =  BeautifulSoup(df['html'][x]).find('div', class_ = 't-14 mt5').text.strip().split(sep='\n')[0]
  except:
    continue



  0%|          | 0/998 [00:00<?, ?it/s]

###Требуемые хард скилы


In [ ]:
# соберем в столбец description описания вакансий в нижнем регистре

df['description'] =[BeautifulSoup(df['html'][x]).find(
    'div', class_='jobs-box__html-content jobs-description-content__text t-14 t-normal jobs-description-content__text--stretch').find('span').text.strip().lower() for x in tqdm(range(len(df['html'])))]

  0%|          | 0/998 [00:00<?, ?it/s]

In [ ]:
df['description'] = df['description'].str.split(pat=' ')


In [ ]:
# соберем в столбец description_2 описания раздела Skills в карточках вакансий

df['description_2'] = np.nan

for x in tqdm(range(len(df['html']))):
  try:
    df['description_2'][x] = BeautifulSoup(df['html'][x]).find('div', class_='mt5 mb2').find_all('span')[-1].text.split(sep=':')[1].strip().split(sep=', +')[0].lower()
  except:
    continue

df['description_2'] = df['description_2'].fillna('n/a')


  0%|          | 0/998 [00:00<?, ?it/s]

In [ ]:
df['description_2'] = df['description_2'].str.split(pat=' ')

In [ ]:
# создадим список хард-скиллов

skills = (['a/b testing', 'ab testing', 'actian', 'adobe analytics', 'adobe audience manager',
    'adobe experience platform', 'adobe launch', 'adobe target', 'airflow',
    'alooma', 'alteryx', 'amazon machine learning', 'amazon web services', 'aml',
    'amplitude', 'ansible', 'apache camel', 'apache nifi', 'apache spark',
    'api', 'asana', 'auth0', 'aws', 'aws glue', 'azure', 'azure data factory',
    'basecamp', 'bash', 'beats', 'big query', 'bigquery', 'birst', 'bitbucket',
    'blendo', 'bootstrap', 'business objects bi', 'c#', 'c++', 'caffe', 'cassandra',
    'cdata sync', 'chronograf', 'ci/cd', 'cicd', 'clickhouse', 'cloudera', 'cluvio',
    'cntk', 'cognos', 'composer', 'computer vision', 'conda', 'confluence',
    'couchbase', 'css', 'd3.js', 'dash', 'dashboard', 'data factory', 'data fusion',
    'data mining', 'data studio', 'data warehouse', 'databricks', 'dataddo',
    'dataflow', 'datahub', 'dataiku', 'datastage', 'dbconvert', 'dbeaver', 'dbt',
    'deep learning', 'dl/ml', 'docker', 'domo', 'dune', 'dv360', 'dynamodb',
    'elasticsearch', 'elt', 'erwin', 'etl', 'etleap', 'excel', 'facebook business manager',
    'fivetran', 'fuzzy', 'ga360', 'gcp', 'gensim', 'ggplot', 'git', 'github', 'gitlab',
    'google ads', 'google analytics', 'google cloud platform', 'google data flow',
    'google optimize', 'google sheets', 'google tag manager', 'google workspace',
    'grafana', 'hadoop', 'hana', 'hanagrafana', 'hbase', 'hdfs', 'hevo data', 'hightouch',
    'hive', 'hivedatabricks', 'html', 'hubspot', 'ibm coremetrics', 'inetsoft',
    'influxdb', 'informatica', 'integrate.io', 'iri voracity', 'izenda', 'java',
    'java script', 'javascript', 'jenkins', 'jira', 'jmp', 'julia', 'jupyter',
    'k2view', 'kafka', 'kantar', 'kapacitor', 'keras', 'kibana', 'kubernetes',
    'lambda', 'linux', 'logstash', 'looker', 'lstm', 'luidgi', 'matillion', 'matlab',
    'matplotlib', 'mendix', 'metabase', 'microsoft sql', 'microsoft sql server',
    'microstrategy', 'miro', 'mixpanel', 'ml', 'ml flow', 'mlflow', 'mongodb', 'mxnet',
    'mysql', 'natural nanguage processing', 'neo4j', 'nlp', 'nltk', 'nosql', 'numpy',
    'oauth', 'octave', 'omniture', 'omnituregitlab', 'openshift', 'openstack',
    'optimizely', 'oracle', 'oracle business intelligence', 'oracle data integrator',
    'pandas', 'panorama', 'pentaho', 'plotly', 'postgre', 'postgresql', 'posthog',
    'power amc', 'power bi', 'power point', 'powerbi', 'powerpivot', 'powerpoint',
    'powerquery', 'pyspark', 'python', 'pytorch', 'pytorchhevo data', 'qlik',
    'qlik sense', 'qlikview', 'querysurge', 'r', 'raphtory', 'rapidminer', 'redash',
    'redis', 'redshift', 'retool', 'rivery', 'rust', 's3', 'sa360', 'salesforce', 'sap',
    'sap business objects', 'sas', 'sas visual analytics', 'scala', 'scikit-learn',
    'scipy', 'seaborn', 'segment', 'selenium', 'sem rush', 'semrush', 'shell', 'shiny',
    'singer', 'sisense', 'skyvia', 'snowflake', 'spacy', 'spark', 'sparkml', 'splunk',
    'spotfire', 'spreadsheet', 'spss', 'sql', 'ssis', 'sssr', 'stambia', 'statistics',
    'statsbot', 'stitch', 'streamlit', 'streamsets', 'svn', 't-sql', 'tableau', 'talend',
    'targit', 'tealium', 'telegraf', 'tensorflow', 'terraapi', 'terraform', 'theano',
    'thoughtspot', 'timeseries', 'trello', 'unix', 'vba', 'vtom', 'webfocus', 'wfh',
    'xplenty', 'xtract.io', 'yellowfin'
          ])

In [ ]:
# создадим столбец с хард-скиллами, взяв их из нашего списка хард-скилло и проверив нахождение каждого в столбцах description и description_2

res = []

for index in tqdm(range(len(df['html']))):
  skills_list = []
  for skill in skills:
    if skill in df['description'][index]:
      skills_list.append(skill)
    elif skill in df['description_2'][index]:
      skills_list.append(skill)

  res.append(skills_list)

df['skills'] = res


  0%|          | 0/998 [00:00<?, ?it/s]

###Время публикации вакансии

In [ ]:
# создадим столбец date, собрав в него данные по времени публикации вакансий

df['date'] = [' '.join(BeautifulSoup(df['html'][x]).find('span', class_='jobs-unified-top-card__posted-date').text.strip().split()[0:2]) for x in tqdm(range(len(df['html'])))]

  0%|          | 0/998 [00:00<?, ?it/s]

In [ ]:
# посмотрим на уникальные значения столбца date

df['date'].unique()

array(['1 week', '2 weeks', '6 days', '3 weeks', '2 days', '1 day',
       '4 days', '4 weeks', '3 days', '5 days', '12 minutes',
       '29 minutes', '5 hours', '8 hours', '6 hours', '9 hours',
       '11 hours', '12 hours', '7 hours', '10 hours'], dtype=object)

In [ ]:
# определим текущую дату как дату сбора данных из источника, взяв ее из названия файла

date = datetime.strptime('23.05.2023', '%d.%m.%Y')

In [ ]:
# вычислим дату публикации вакансии как разницу между относительной текущей датой и данными столбца date

df['dt'] = np.nan

for x in tqdm(range(len(df['html']))):
  if 'week' in df['date'][x] or 'weeks' in df['date'][x]:
    df['dt'][x] = (date - timedelta(weeks=int(df['date'][x][0]))).strftime('%d.%m.%Y')
  elif 'day' in df['date'][x] or 'days' in df['date'][x]:
    df['dt'][x] = (date - timedelta(days=int(df['date'][x][0]))).strftime('%d.%m.%Y')
  else:
    df['dt'][x] = date.strftime('%d.%m.%Y')



  0%|          | 0/998 [00:00<?, ?it/s]

###Количество кандидатов на вакансию

In [ ]:
df['applicants'] = np.nan


for x in tqdm(range(len(df['html']))):
  try:
    df['applicants'][x] = BeautifulSoup(df['html'][x]).find('div', class_='jobs-unified-top-card__primary-description').find('span', class_='jobs-unified-top-card__applicant-count').text.strip().split()[0]
  except:
    continue

  0%|          | 0/998 [00:00<?, ?it/s]

###Формирование итогового датасета

In [ ]:
# сохраним в датасете только нужные нам данные

df_final = df[['name', 'city', 'country',
       'employment_type', 'company_title', 'company_size', 'industry',
       'skills', 'dt', 'applicants']]

df_final = df_final.rename(columns={'Unnamed: 0':'id'})

###Вывод



Мы распарсили предоставленный csv файл и создали следующие
признаки:

 - наименование вакансии
 - город;
 - страна;
 - тип занятости;
 - компания;
 - размер компании (количество работников);
 - сфера деятельности компании;
 - требуемые хард скилы;
 - дата публикации вакансии;
 - количество кандидатов на вакансию.

Данные готовы к предобработке.

##Предобработка данных

In [ ]:
# заменим пустые ячейки в столбце country на nan

for x in tqdm(range(len(df_final))):
  if df_final['country'][x] == '':
    df_final['country'][x] = np.nan

  0%|          | 0/998 [00:00<?, ?it/s]

In [ ]:
for x in tqdm(range(len(df['html']))):
  df_final['skills'][x] = ' '.join(df_final['skills'][x]).replace(' ', ', ')

  0%|          | 0/998 [00:00<?, ?it/s]

In [ ]:
# посчитаем дубликаты

df_final.duplicated().sum()

112

In [ ]:
# удалим дубликаты

df_final = df_final.drop_duplicates().reset_index(drop = True)

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 886 entries, 0 to 885
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             886 non-null    object
 1   city             886 non-null    object
 2   country          836 non-null    object
 3   employment_type  819 non-null    object
 4   company_title    875 non-null    object
 5   company_size     865 non-null    object
 6   industry         853 non-null    object
 7   skills           886 non-null    object
 8   dt               886 non-null    object
 9   applicants       727 non-null    object
dtypes: object(10)
memory usage: 69.3+ KB


In [ ]:
# приведем столбец с датой вакансии к формату datetime

df_final['dt'] = pd.to_datetime(df_final['dt'],  format='%d.%m.%Y')

In [ ]:
# приведем столбец с числом соискателей к формату float

df_final['applicants'] = df_final['applicants'].astype('float')

In [ ]:
# изучим уникальные значения столбца company_size

df_final['company_size'].value_counts()

10,001+         217
1,001-5,000     183
51-200          165
201-500          86
501-1,000        67
5,001-10,000     59
11-50            53
1-10             25
See               7
Retail            1
Svein             1
Romain            1
Name: company_size, dtype: int64

In [ ]:
# заменим некорретные значения столбца company_size с размером компании на Nan

df_final.loc[df_final['company_size'] == 'See', 'company_size'] = np.nan
df_final.loc[df_final['company_size'] == 'Retail', 'company_size'] = np.nan
df_final.loc[df_final['company_size'] == 'Romain', 'company_size'] = np.nan
df_final.loc[df_final['company_size'] == 'Svein', 'company_size'] = np.nan

In [ ]:
# изучим уникальные значения городов

df_final['city'].unique()

array(['Basel', 'Coventry', 'South Molton', 'Lugano', 'Southampton',
       'Leeds', 'Nuneaton', 'Paris', 'Cambridge', 'West Midlands',
       'Schiphol', 'Chester', 'Craven Arms', 'Dublin', 'Belfast',
       'Sunderland', 'Montévrain', 'Bristol', 'Solihull', 'Blackpool',
       'Cracow', 'Dijon', 'Alsónémedi', 'Manchester',
       'Elliniko-Argyroupoli', 'Italy', 'Umeå', 'North Holland',
       'Vilnius', 'Durham', 'Oudenaarde', 'Milan', 'Stockholm County',
       'Luxembourg', 'Roubaix', 'Munich', 'Zaventem',
       'Brussels Metropolitan Area', 'Portugal', 'West Malling',
       'Greater Paris Metropolitan Region', 'France', 'Bulgaria', 'Lille',
       'Egham', 'Karlstad', 'Madrid', 'Barcelona', 'Oxford', 'Hungary',
       'Brussels', 'Taibon', 'Epsom', 'Amsterdam', 'Spinea',
       'Greater Palma de Mallorca Metropolitan Area', 'Brindisi',
       'Boulogne-Billancourt', 'Wolfsburg', 'Nantes', 'Derby', 'Lund',
       'Garwolin', 'Stockholm', 'Rome', 'Massy', 'Prague',
       'Middle

In [ ]:
# удалим названия стран из списка городов

for city in tqdm(df_final['city'].unique()):
  for country in countries:
    df_final.loc[df_final['city'] == country, 'city'] = np.nan

  0%|          | 0/428 [00:00<?, ?it/s]

In [ ]:
# отфильтруем датасет, оставив в нем только вакансии аналитиков, по корню слова analyst

res =[]


for x in tqdm(range(len(df_final))):
  if 'Anal' in df_final['name'][x] or 'DA' in df_final['name'][x]:
    res.append(True)
  else:
    res.append(False)

df_final['relevance'] = res

df_final = df_final[df_final['relevance'] == True].reset_index(drop=True).drop(columns='relevance')


  0%|          | 0/886 [00:00<?, ?it/s]

In [ ]:
# выделим в отдельный столбик вакансии для джунов

res =[]

for x in tqdm(range(len(df_final))):
  if 'Jun' in df_final['name'][x]:
    res.append(True)
  else:
    res.append(False)

df_final['junior_status'] = res



  0%|          | 0/631 [00:00<?, ?it/s]

In [ ]:
# выделим в отдельном столбце вакансии для bi аналитиков

res =[]

for x in tqdm(range(len(df_final))):
  if 'BI' in df_final['name'][x] or 'Intelligence' or 'business intelligence analyst' in df_final['name'][x]:
    res.append(True)
  else:
    res.append(False)

df_final['bi_status'] = res

  0%|          | 0/631 [00:00<?, ?it/s]

In [ ]:
# выделим в отдельном столбце вакансии для data аналитиков

res =[]

for x in tqdm(range(len(df_final))):
  if 'Data Analyst' in df_final['name'][x]:
    res.append(True)
  else:
    res.append(False)

df_final['data_analyst_status'] = res

  0%|          | 0/631 [00:00<?, ?it/s]

In [ ]:
# проставим id каждой вакансии


df_final['id'] = range(0, len(df_final))

In [ ]:
# преобразуем перечень навыков в списки

df_final['skills'] = df_final['skills'].str.split(pat=',')


In [ ]:
# развернем столбец skills так, чтобы на каждый навык приходилась одна строка

df_final = df_final.explode(
    column="skills", ignore_index=True
)

In [ ]:
# создадим столбец, в котором отметим все профессиональные направления вакансий

def type_vac(row):
  if row['junior_status'] == True:
    return 'Junior'
  if row['bi_status'] == True:
    return 'BI'
  if row['data_analyst_status'] == True:
    return 'DA'
  if row['data_analyst_status'] == True and row['junior_status'] == True:
    return 'DA'
  if row['bi_status'] == True and row['junior_status'] == True:
    return 'BI'
  else:
    return 'Other'



In [ ]:
df_final['vacancy_status'] = df_final.apply(type_vac, axis=1)

In [ ]:
df_final

,name,city,country,employment_type,company_title,company_size,industry,skills,dt,applicants,junior_status,bi_status,data_analyst_status,id,vacancy_status
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50,Staffing & Recruiting,sap,2023-05-16,47.0,False,True,True,0,BI
1,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50,Staffing & Recruiting,sas,2023-05-16,47.0,False,True,True,0,BI
2,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50,Staffing & Recruiting,statistics,2023-05-16,47.0,False,True,True,0,BI
3,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,NaN,NaN,,2023-05-16,NaN,False,True,True,1,BI
4,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,NaN,NaN,excel,2023-05-16,NaN,False,True,True,2,BI
5,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,NaN,NaN,etl,2023-05-09,NaN,False,True,True,3,BI
6,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,NaN,NaN,oracle,2023-05-09,NaN,False,True,True,3,BI
7,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,NaN,NaN,python,2023-05-09,NaN,False,True,True,3,BI
8,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,NaN,NaN,sql,2023-05-09,NaN,False,True,True,3,BI
9,Data Analyst - Logistics,Southampton,United Kingdom,On-site,"Butler, Bridge & May",NaN,NaN,excel,2023-05-17,NaN,False,True,True,4,BI


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1074 entries, 0 to 1073
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   name                 1074 non-null   object        
 1   city                 1001 non-null   object        
 2   country              1020 non-null   object        
 3   employment_type      947 non-null    object        
 4   company_title        1066 non-null   object        
 5   company_size         1035 non-null   object        
 6   industry             1031 non-null   object        
 7   skills               1074 non-null   object        
 8   dt                   1074 non-null   datetime64[ns]
 9   applicants           823 non-null    float64       
 10  junior_status        1074 non-null   bool          
 11  bi_status            1074 non-null   bool          
 12  data_analyst_status  1074 non-null   bool          
 13  id                   1074 non-nul

###Вывод

В ходе предобработки данных мы:

- Адаптировали форматы данных столбцов;
- Удалили дубликаты;
- Изучили уникальные значения столбцов и исправили неверные значения;
- Создали дополнительные признаки: отметили вакансии для джуниров, для BI специалистов и дата аналитиков;
- Проставили id для каждой вакансии;
- "Развернули" столбец со скилами, чтобы на каждый навык приходилась одна строка.

##Общий вывод

В ходе первого этапа работы мы:

- Распарсили предоставленный csv файл и создали следующие признаки:

    - наименование вакансии
    - город;
    - страна;
    - тип занятости;
    - компания;
    - размер компании (количество работников);
    - сфера деятельности компании;
    - требуемые хард скилы;
    - дата публикации вакансии;
    - количество кандидатов на вакансию.

- Адаптировали форматы данных столбцов;
- Удалили дубликаты;
- Изучили уникальные значения столбцов и исправили неверные значения;
- Создали дополнительные признаки: отметили вакансии для джуниров, для BI - специалистов и дата аналитиков;
- Проставили id для каждой вакансии;
- "Развернули" столбец со скилами, чтобы на каждый навык приходилась одна строка.

Данные готовы к визуализации.

In [ ]:
# выгрузим данные в таблицу csv

df_final.to_csv('linkedin_data.csv')

##Дашборд

Ссылка на дашборд: https://datalens.yandex/g4kei0mo53ao6
